
___
# Sistema de Recomendação de Filmes- Filtragem Colaborativa

Agora vamos utilizar o conceito de filtragem colaborativa para fazer recomendações de filmes.


crédito/fonte: https://github.com/krishnaik06

In [ ]:
import numpy as np
import pandas as pd

#### Leitura dos dados

In [ ]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [ ]:
df.head()

títulos dos filmes:

In [ ]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

In [ ]:
movie_titles.loc[5,'title']

In [ ]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

# Exploração de Dados


## Visualização

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

### Vamos explorar o comando 'groupby' do Pandas para verificar os valores médios das notas e a quantidade de avaliações. 

In [ ]:
df.groupby('title')['rating'].mean().sort_values(ascending=False).head()

In [ ]:
df.groupby('title')['rating'].count().sort_values(ascending=False).head()

In [ ]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings.head()

## Adicionando a coluna com a quantidade de avaliações:

In [ ]:
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head()

## - Histogramas para avaliar a quantidade de ratings e as notas médias:

In [ ]:
plt.figure(figsize=(10,4))
ratings['num of ratings'].hist(bins=70)

In [ ]:
plt.figure(figsize=(10,4))
ratings['rating'].hist(bins=70)

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

## Recomendação de filmes similares

### Agora vamos criar a Matriz com as notas dos filmes por usuários, através do método 'pivot_table' do Pandas.

In [ ]:
moviemat = df.pivot_table(index='user_id',columns='title',values='rating')
moviemat.head(5)

### Filmes com mais avaliações

In [ ]:
ratings.sort_values('num of ratings',ascending=False).head(10)

### Vamos fazer um teste com dois filmes: 'Star Wars' e 'Liar Liar'.

In [ ]:
ratings.head()

### Capturando da matriz com as notas dos usuários por filmes, as notas dos filmes escolhidos.

In [ ]:
starwars_user_ratings = moviemat['Star Wars (1977)']
liarliar_user_ratings = moviemat['Liar Liar (1997)']
starwars_user_ratings.head()

### Vamos buscar os filmes com avaliações similares através da correlação entre as séries utilizando o método 'corrwith' do Pandas:

In [ ]:
similar_to_starwars = moviemat.corrwith(starwars_user_ratings)

### Removendo valores NaN do resulato:

In [ ]:
corr_starwars = pd.DataFrame(similar_to_starwars,columns=['Correlation'])
corr_starwars.dropna(inplace=True)
corr_starwars.head()

Vamos ordenar os dados resultantes para encontrar os valores com a maior correlação. 

In [ ]:
corr_starwars.sort_values('Correlation',ascending=False).head(10)

### - Verificação da quantidade de avaliações dos filmes.

In [ ]:
corr_starwars = corr_starwars.join(ratings['num of ratings'])
corr_starwars.head()

### Vamos agora selecionar somente os filmes que tem pelo menos 100 avaliações:

In [ ]:
corr_starwars[corr_starwars['num of ratings']>100].sort_values('Correlation',ascending=False).head()

## Utilizando o modelo para o filme 'Liar Liar'.

In [ ]:
similar_to_liarliar = moviemat.corrwith(liarliar_user_ratings)
corr_liarliar = pd.DataFrame(similar_to_liarliar,columns=['Correlation'])
corr_liarliar.dropna(inplace=True)
corr_liarliar = corr_liarliar.join(ratings['num of ratings'])
corr_liarliar[corr_liarliar['num of ratings']>100].sort_values('Correlation',ascending=False).head()